In [1]:
import cv2 as cv
from glob import glob
import os
import numpy as np
from utils.poincare import calculate_singularities
from utils.segmentation import create_segmented_and_variance_images
from utils.normalization import normalize
from utils.gabor_filter import gabor_filter
from utils.frequency import ridge_freq
from utils import orientation
from utils.crossing_number import calculate_minutiaes
from utils.skeletonize import skeletonize

from MyMAS import MAS

/mnt/sdb3/TEST/fingerprint_recognition/venv/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def fingerprint_pipline(input_img):
    block_size = 16

    # pipe line picture re https://www.cse.iitk.ac.in/users/biometrics/pages/111.JPG
    # normalization -> orientation -> frequency -> mask -> filtering

    # normalization - removes the effects of sensor noise and finger pressure differences.
    normalized_img = normalize(input_img.copy(), float(100), float(100))

    # color threshold
    # threshold_img = normalized_img
    # _, threshold_im = cv.threshold(normalized_img,127,255,cv.THRESH_OTSU)
    # cv.imshow('color_threshold', normalized_img); cv.waitKeyEx()

    # ROI and normalisation
    (segmented_img, normim, mask) = create_segmented_and_variance_images(normalized_img, block_size, 0.2)

    # orientations
    angles = orientation.calculate_angles(normalized_img, W=block_size, smoth=False)
    orientation_img = orientation.visualize_angles(segmented_img, mask, angles, W=block_size)
    
    # find the overall frequency of ridges in Wavelet Domain
    freq = ridge_freq(normim, mask, angles, block_size, kernel_size=5, minWaveLength=5, maxWaveLength=15)

    # create gabor filter and do the actual filtering
    gabor_img = gabor_filter(normim, angles, freq)

    # thinning oor skeletonize
    thin_image = skeletonize(gabor_img)

    # minutias
    minutias, end, bif = calculate_minutiaes(thin_image)

    # singularities
    singularities_img, FingerType = calculate_singularities(thin_image, angles, 1, block_size, mask)

    # visualize pipeline stage by stage
#     output_imgs = [input_img, normalized_img, segmented_img, orientation_img, gabor_img, thin_image, minutias, singularities_img]
    output_imgs = [input_img, minutias, singularities_img]
    
#     for i in range(len(output_imgs)): # Lặp qua từng ảnh output
#         if len(output_imgs[i].shape) == 2: # Nếu ảnh thứ i trong output là ảnh Gray(có 2 chiều)
#             output_imgs[i] = cv.cvtColor(output_imgs[i], cv.COLOR_GRAY2RGB) # Chuyển thành ảnh RGB để concatenate với 2 ảnh cuối
    
    # Chuyển list các ảnh thành ảnh 4x4
#     results = np.concatenate([np.concatenate(output_imgs[:4], 1), np.concatenate(output_imgs[4:], 1)]).astype(np.uint8)

#     return results
    return output_imgs, end, bif, FingerType

In [3]:
def DISTANCE(U, V) -> float:
    """
    """
    return np.sqrt((U[0] - V[0])**2 + (U[1] - V[1])**2)

In [4]:
def Points_distance(points1, points2) -> list:
    n = points1.shape[0]
    m = points2.shape[0]
    Distance = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            Distance[i, j] = DISTANCE(points1[i], points2[j])
    return Distance

In [5]:
def Filter_Points(ListPoints, CenterPoint, size = (0,0)):
    """
    """
    n = ListPoints.shape[0]
    ResultPoints = []
    for i in range(n):
        distance = np.abs(CenterPoint - ListPoints[i])
        if (distance[0] <= size[0]/2) and (distance[1] <= size[1]/2):
            ResultPoints.append(ListPoints[i].tolist())
    return np.array(ResultPoints)

In [6]:
def Transtation_Finger(Minutiae, Trans_Vector) -> np.ndarray:
    """
    """
    return Minutiae + Trans_Vector

In [7]:
def MATCH(finger1, finger2, alpha=0.5, box_size=(0, 0)) -> bool:
    """
    """
    # Get feature
    center_point1, end_1, bif_1 = finger1[0], finger1[1], finger1[2]
    center_point2, end_2, bif_2 = finger2[0], finger2[1], finger2[2]
    
    translation_vector = center_point2 - center_point1
    
    # Translation minutiaes
    end_1 = Transtation_Finger(end_1, translation_vector)
    bif_1 = Transtation_Finger(bif_1, translation_vector)
    
    # Filter minutiaes
    new_end_1 = Filter_Points(end_1, center_point2, size=box_size)
    new_bif_1 = Filter_Points(bif_1, center_point2, size=box_size)
    new_end_2 = Filter_Points(end_2, center_point2, size=box_size)
    new_bif_2 = Filter_Points(bif_2, center_point2, size=box_size)
    
    n1_end = new_end_1.shape[0]
    n1_bif = new_bif_1.shape[0]
    n2_end = new_end_2.shape[0]
    n2_bif = new_bif_2.shape[0]
    
    # Match ending points
    Distance_end = Points_distance(new_end_1, new_end_2)
    end_min_dist = Distance_end.min(axis=0)
    end_min_index = Distance_end.argmin(axis=0)
    
    # Match bifucation points
    Distance_bif = Points_distance(new_bif_1, new_bif_2)
    bif_min_dist = Distance_bif.min(axis=0)
    bif_min_index = Distance_bif.argmin(axis=0)
    
    # Check Matching
    end_matched = np.where(end_min_dist <= 15)[0].shape[0]
    bif_matched = np.where(bif_min_dist <= 15)[0].shape[0]
    
    if (end_matched + bif_matched)/(n1_end + n1_bif) >= alpha:
        return True
    else:
        return False

In [8]:
box_size = (150, 300)

In [9]:
# MATCH(images[0], images[2], box_size=(150, 300))

In [10]:
def Draw_Result():
    """
    """
    pass

In [11]:
def FeaturesExtraction(finger):
    """
    """
    imgresult, end, bif, fingertype = fingerprint_pipline(finger)
    center_point = np.array(fingertype)[:,1:].astype(int).mean(axis=0)
    
    end = np.array(end)[:,1:]
    bif = np.array(bif)[:,1:]
    
    return [center_point, end, bif]

In [12]:
# finger1 = FeaturesExtraction(images[0])
# finger2 = FeaturesExtraction(images[3])

In [13]:
# MATCH(finger1, finger2, box_size=(150,300))

In [14]:
def S_Generation(PIN, lenMask) -> str:
    """
    """
    S = bin(PIN)[2:]
    n = len(S)
    k = lenMask//n
    
    return S*k + S[:lenMask - n*k]
    

In [15]:
def Features2Msg(features):
    """
    """
    center, end, bif = features[0].astype(int), features[1].astype(int), features[2].astype(int)
    
    center_msg = "{:3d}{:3d}".format(center[0], center[1]).replace(' ', '0')
    
    n = end.shape[0]
    m = bif.shape[0]
    
    end_msg = ''
    for point in end:
        end_msg += "{:3}{:3}".format(point[0], point[1]).replace(' ', '0')
    
    bif_msg = ''
    for point in bif:
        bif_msg += "{:3}{:3}".format(point[0], point[1]).replace(' ', '0')
    
    return "{}{:3}{:3}{}{}".format(center_msg, n, m, end_msg, bif_msg).replace(' ', '0')
    

In [16]:
#Features2Msg(finger1)

In [17]:
def EncryptFinger(finger, PIN, A, B, X, Code, lenMask, k, padWord):
    """
    """
    # generation S
#     S = S_Generation(PIN, lenMask)
    S = b'10001101101011101000100011011010111010001110100011101000'
    print('generation Mask successful!')
    
    Msg = Features2Msg(finger)
    print('generation Msg successful!')
    
    cryptosystem = MAS(A, B, Code, X, S, k, padWord)
    
    EMsg = cryptosystem.Encode(Msg)
    print('Encode successful!')
    
    return EMsg

In [18]:
def Emsg2Features(EMsg, PIN, A, B, X, Code, lenMask, k, padWord):
    """
    """
    end = []
    bif = []
    
    S = b'10001101101011101000100011011010111010001110100011101000'
    print('generation Mask successful!')
    
    cryptosystem = MAS(A, B, Code, X, S, k, padWord)
    
    Result =  "".join(cryptosystem.Decode(EMsg))
    
    CenterPoint = np.array([Result[:3], Result[3:6]]).astype(int)
    n, m = int(Result[6:9]), int(Result[9: 12])
    
    t = 12
    for i in range(n):
        end.append([int(Result[t: t+3]), int(Result[t+3: t+6])])
        t += 6
    for i in range(m):
        bif.append([int(Result[t: t+3]), int(Result[t+3: t+6])])
        t += 6
    return [np.array(CenterPoint), np.array(end), np.array(bif)]

In [19]:
# PIN = 999999

# S = S_Generation(PIN, 56)
# print(S, '\n', len(S))

In [20]:
# Config Cryptosystem
PIN = 123456
lenMask = 56

# Example Initialization
A = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
B = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n','o', 'p']
Code = {'a': b'1000',
        'b': b'1110',
        'c': b'0011', 
        'd': b'1111', 
        'e': b'1101', 
        'f': b'0010', 
        'g': b'1100', 
        'h': b'0101',
        'i': b'1011',
        'j': b'0000',
        'k': b'1001',
        'l': b'0111',
        'm': b'0100',
        'n': b'1010',
        'o': b'0001',
        'p': b'0110'}

# Language
X0 = ['a', 'cgh']
X1 = ['egm', 'nmc']
X2 = ['ig', 'fce']
X3 = ['jkd']
X4 = ['bea', 'mok']
X5 = ['fno', 'ihc']
X6 = ['cei']
X7 = ['demc', 'khm']
X8 = ['lbkh']
X9 = ['kog', 'dcef']
X = [X0, X1, X2, X3, X4, X5, X6, X7, X8, X9]
paddingWord = 'p'
k = 3

In [53]:
finger_path1 = './sample_inputs/101_1.tif'

image1 = cv.imread(finger_path1, 0)

finger_origin = FeaturesExtraction(image1)

In [50]:
EMsg = EncryptFinger(finger_origin, PIN, A, B, X, Code, lenMask, k, paddingWord)

generation Mask successful!
generation Msg successful!
MAS cryptosystem initialization successful!
Author: Thanh HoangVan
Github: thanhhoangvan
+-----------------------------------------+
Encode successful!


In [51]:
finger_saved = Emsg2Features(EMsg, PIN, A, B, X, Code, lenMask, k, paddingWord)

generation Mask successful!
MAS cryptosystem initialization successful!
Author: Thanh HoangVan
Github: thanhhoangvan
+-----------------------------------------+


In [58]:
finger_path2 = './sample_inputs/101_3.tif'
image2 = cv.imread(finger_path2, 0)
finger_match = FeaturesExtraction(image2)

In [59]:
MATCH(finger_saved, finger_match, box_size=(150, 300))

False